
##### Note : this notebook contains the week 3 project assignment of the course 'Applied Data Science Capstone'   Coursera 

<h1 style="color: blue;"> 1.Scraping the Dataset of Toronto Neighborhood<h1>

The dataset of the Toronto neighborhood is not available in csv form for manipulation, therefore its data should be scraped from wikipedia here is the technique with step by step guide of how to scrap the data.
The Beautiful Soup package is used for scraping the data.

Before getting start lets download the following libraries

In [1]:
from bs4 import BeautifulSoup # for scraping data from wikipedia
import requests #calling request to url
import pandas as pd # handling data frame
import numpy as np
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import geocoder # getting latitude and longitude
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm # ploting graphs
import matplotlib.colors as colors
from sklearn.cluster import KMeans #for implementing kmeans algorithm
import folium # ploting map

 Getting the request and parsing 

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XpVnAwpAAEYAAEsuV0QAAADD","wgCSPNonce":!1,"wgCanonica

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(url).text
soup=BeautifulSoup(source,'lxml')


If you carefully inspect the HTML script all the table contents i.e. postalcode,borough and neighborhood of the toronto which we intend to extract is under class Wikitable .

In [21]:
my_table=soup.find('table',{'class':'wikitable'})#find the class where required data is place


Now after examing the data we sure our required data lies in a step from 'tbody','tr' to the element of 'td'.Creating a list for storing our data.

In [22]:

data=[]
table_body=my_table.find('tbody')
sub_table=table_body.find_all('tr')
for row in sub_table:
    sub=row.find_all('td') # data resides in the element of td
    sub=[ele.text.strip() for ele in sub]
    data.append(sub)



### Transform the data into Pandas dataframe  

From the links, we have to extract our required data which is postalcode,borough and neighborhood.
Now Convert the following data into Pandas DataFrame to work in python.


In [23]:
arr=np.array(data)
arr_refine=[x for x in arr if x] #removing the empty list

postal_code=[index[0] for index in arr_refine] #getting the postalcode
borough=[index[1] for index in arr_refine] #getting the borough
neighbor=[index[2] for index in arr_refine] #getting the neighorhood


 
df = pd.DataFrame(list(zip(postal_code,borough,neighbor)), #creating a table of following columns
               columns =['Postal Code','Borough','Neighborhood'])


df_refine=df[df.Borough !='Not assigned'].reset_index(drop=True) # as required ignore the Not assigned cells

toronto_data=df_refine.apply(lambda x: x.str.replace('/',',')) #as required add comma

toronto_data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


For calculating row lenght of our data

In [24]:
toronto_data.shape[0]

103

<h1 style="color: blue;"> 2. Adding the Location(Latitude & Longitude) in Dataset <h1>

Here the csv file data set location is used you can get it from Link :http://cocl.us/Geospatial_data

In [25]:
location=pd.read_csv('Geospatial_Coordinates.csv')
toronto_data_location=pd.merge(toronto_data,location, on=['Postal Code']) #Alert on merging columns name should be same
toronto_data_location

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h1 style="color: blue;"> 3. Implement the Clustering<h1>
 

In this section explore the neighborhood of Toronto using foursquare api and implementing the kmeans algorithm for clustering.

### Setting up Foursquare API

In [26]:
CLIENT_ID='R2R05S3T5J04JUU4YNXM5KEELQIH23P3X4131WIP2FPSXWBO'
CLIENT_SECRET='2JHRTJ4IG2AFO4HS3KJLZCAHV5EGIZF1ZBVT55T33IBQOCVG'
VERSION='20180604'
radius=500
LIMIT=50

Defining the latitude and longitude for api setup

In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude {},Longitude{}'.format(latitude,longitude))

In [ ]:
n_name=toronto_data_location.loc[0,'Neighborhood']
n_lat=toronto_data_location.loc[0,'Latitude']

n_lng=toronto_data_location.loc[0,'Longitude']

#### Here create a function for getting call request to api and tranform the result data into clean pandas dataframe

In [ ]:
def NearVenues(n_name, n_lat, n_lng):
    
    venues_list=[]
    for name, lat, lng in zip(n_name, n_lat, n_lng):
        print(name)
            
     
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
       
        results = requests.get(url).json()["response"]['groups'][0]['items']
        

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Remember this cell will take some time .First it print all names of neighborhood to completely give the latitude and longitude 
of all the borough to api.

##### Calling NearVenues  function 

In [ ]:
toronto_venues = NearVenues(n_name=toronto_data_location['Neighborhood'],
                                   n_lat=toronto_data_location['Latitude'],
                                   n_lng=toronto_data_location['Longitude']
                                  )

In [ ]:
toronto_venues.head()

#### Analysis the neighborhood venues

In [ ]:
For manipulating the data we need to deal with numerical form therefore,get_dummies convert categorical data into numbers.

In [ ]:
toronto_refine = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_refine['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_refine.columns[-1]] + list(toronto_refine.columns[:-1])
toronto_refine = toronto_refine[fixed_columns]

toronto_refine.head()

In [ ]:
toronto_group = toronto_refine.groupby('Neighborhood').mean().reset_index() # taking out mean 

###### Here created a new data frame that display the top 10 venues for each neighbor

In [ ]:
top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_group['Neighborhood']

for ind in np.arange(toronto_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], top_venues)

neighborhoods_venues_sorted.head()

## Clustering the Neighborhood

Used the sklearn kmeans library to perform the kmeans algorithm

In [ ]:
kclusters = 5
manhattan_grouped_clustering = toronto_group.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)
kmeans.labels_[0:10] 

Here created a new data frame that cluster and add top 10 venues for each neighborhood

In [ ]:
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_, allow_duplicates = False)
toronto_merged = toronto_data_location
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [ ]:
toronto_merged.head()

#### Visualizing data in graphical form using matplot and folium libraries

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [ ]:
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###### Thanks and Welcome to any recommendation.